In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from stuff import *
float_formatter = lambda x: "%.2f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})



In [ ]:
iters = 400
plt.figure(figsize=(12,5))

model = Model(nodes=4, dim=15)

mu_x, mu_xy, L_x, L_xy = model.get_mu_L()

plt.subplot(121)
C = 1  # big O const
N = np.arange(iters)
chi = (L_x / mu_x) * (L_xy / mu_xy) ** 2
plt.plot(np.exp(-N / (C * chi ** 0.5)), label='g acc')
# plt.plot(np.exp(-N / (C * chi)), label='g')

plt.subplot(122)
plt.plot(np.exp(-N / (C * (L_x / mu_x)**0.5 * L_xy / mu_xy)), label='APDM')
# plt.plot(cons_err, label='APDM')


x_res, f_err, cons_err = APDM(iters, model)

plt.subplot(121)
plt.plot(f_err, label='APDM')
plt.title('$|f -f^*|$')
plt.yscale('log')

plt.subplot(122)
plt.plot(cons_err, label='APDM')
plt.title('$||Ax||$')
plt.yscale('log')


x_res, f_err, cons_err = LDM(iters, model)

plt.subplot(121)
plt.plot(f_err, label='locally dual')
plt.legend()

plt.subplot(122)
plt.plot(cons_err, label='locally dual')
plt.legend()


x_res, f_err, cons_err = GDM(iters, model)

plt.subplot(121)
plt.plot(f_err, label='globally dual')
plt.legend()

plt.subplot(122)
plt.plot(cons_err, label='globally dual')
plt.legend()

x_res, f_err, cons_err = GDAM(iters, model)

plt.subplot(121)
plt.plot(f_err, label='globally dual acc')
plt.legend()

plt.subplot(122)
plt.plot(cons_err, label='globally dual acc')
plt.legend()

x_res, f_err, cons_err = LDAM(iters, model)

plt.subplot(121)
plt.plot(f_err, label='locally dual acc')
plt.legend()

plt.subplot(122)
plt.plot(cons_err, label='globally dual acc')
plt.legend()

In [ ]:
iters = 400
plt.figure(figsize=(12,5))

model = Model(nodes=4, dim=15)


params = list(get_apdm_params(model))
x_res, f_err, cons_err = APDM(iters, model, params)


plt.subplot(121)
plt.plot(f_err, label=f'APDM, beta={params[2]}')
plt.title('$|f -f^*|$')
plt.yscale('log')

plt.subplot(122)
plt.plot(cons_err, label=f'APDM, beta={params[2]}')
plt.title('$||Ax||$')
plt.yscale('log')


params = list(get_apdm_params(model))
params[2] *= 0.7
x_res, f_err, cons_err = APDM(iters, model, params)

plt.subplot(121)
plt.plot(f_err, label=f'APDM, beta={params[2]}')
plt.legend()

plt.subplot(122)
plt.plot(cons_err, label=f'APDM, beta={params[2]}')
plt.legend()